## 作業目標: 了解斷詞演算法的背後計算

### 根據課程講述的內容, 請計算出下列剩餘所有情況的機率
若有一個人連續觀察到三天水草都是乾燥的(Dry), 則這三天的天氣機率為何？
(Hint: 共有8種可能機率)

```python
states = ('sunny', 'rainy')
observations = ('dry', 'damp', 'soggy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
emission_probatility = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}
```

In [1]:
states = ('sunny', 'rainy')
observations = ('dry', 'damp', 'soggy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
emission_probatility = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}

In [2]:
from itertools import product
mylist=list(product(['sunny','rainy'],repeat=3)) #list all possible event combination

In [3]:
"""
觀察狀態 = 'dry', 'dry', 'dry'
最大機率為: Sunny, Sunny, Sunny
"""
for i in range(len(mylist)):
    state_list=mylist[i]
    day1_w=mylist[i][0]
    day2_w=mylist[i][1]
    day3_w=mylist[i][2]
    probability=\
    start_probability[day1_w]*emission_probatility[day1_w]['dry']*\
    transition_probability[day1_w][day2_w]*emission_probatility[day2_w]['dry']*\
    transition_probability[day2_w][day3_w]*emission_probatility[day3_w]['dry']
    print("The probability of {} is {:.6f}".format(state_list,probability))

The probability of ('sunny', 'sunny', 'sunny') is 0.031104
The probability of ('sunny', 'sunny', 'rainy') is 0.003456
The probability of ('sunny', 'rainy', 'sunny') is 0.001728
The probability of ('sunny', 'rainy', 'rainy') is 0.000672
The probability of ('rainy', 'sunny', 'sunny') is 0.003888
The probability of ('rainy', 'sunny', 'rainy') is 0.000432
The probability of ('rainy', 'rainy', 'sunny') is 0.000756
The probability of ('rainy', 'rainy', 'rainy') is 0.000294


### 根據上述條件, 寫出Viterbi應用程式

In [4]:
observations = ('dry', 'dry', 'dry') #實際上觀察到的狀態為dry, dry, dry
states = ('sunny', 'rainy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
emission_probatility = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}

In [122]:
"""
my code
"""
def viterbi(obs,states, start_p, trans_p,emis_p):
    from itertools import product
    import numpy as np
    def multiplyList(myList) :     
        result = 1
        for x in myList:
             result = result * x  
        return result 
    
    path_list=list(product(states,repeat=len(obs))) #list all possible event combination
    V = [] 
    for path in path_list:
        p1=round(start_p[path[0]]*emis_p[path[0]][obs[0]],6)
        path_prob= [p1]                   
        for i in range(1,len(path)):
            p_i= round(transition_probability[path[i-1]][path[i]]*emission_probatility[path[i]][obs[i]],6)
            path_prob.append(p_i)                                                                               
        path_p={path,round(multiplyList(path_prob),6)}
        V.append(path_p) 
            
    return (max(V))

In [123]:
result=viterbi(observations, states, start_probability, transition_probability , emission_probatility)
result

{('sunny', 'sunny', 'sunny'), 0.031104}

In [10]:
"""
Example code
"""
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]  #probability 
    path = {} #dictionary

    # Initialize base cases (t == 0)
    for s in states:
        
        # 假設第一天隱藏狀態是s且觀察狀態是obs[0]的機率
        V[0][s] = start_p[s] * emit_p[s][obs[0]]    # [{'sunny':機率}]
        
        #list_path {'sunny': ['sunny']}-->{'sunny': ['sunny'], 'rainy': ['rainy']}
        path[s] = [s] 

    # Run Viterbi for t > 0
    for t in range(1,len(obs)):
        V.append({})  # append後：[{'sunny': 1}, {}]
        newpath = {}

        for cur_state in states:
            (prob, state) = max([(V[t-1][pre_state] * trans_p[pre_state][cur_state] * emit_p[cur_state][obs[t]], pre_state) for pre_state in states])
            
            V[t][cur_state] = prob
            newpath[cur_state] = path[state] + [cur_state]

        # Don't need to remember the old paths
        path = newpath
    
    (prob, state) = max([(V[len(obs) - 1][final_state], final_state) for final_state in states])
    return (prob, path[state])


result=viterbi(observations, states, start_probability, transition_probability , emission_probatility)
result

(0.031103999999999993, ['sunny', 'sunny', 'sunny'])